In [1]:
### cannot be done in test_ben0 conda environment due to collisions. Instead, I'll make another conda env
### specifically for data viz. I will call it: data_viz0 (lol)
### remember, use test_ben0 for actual inference.

import numpy as np
import mdtraj as md
import nglview as nv

**Raw PDB, visualized:**

In [4]:
pdb_path = "/home/bfd21/rds/hpc-work/sample_cyclic_md/ligand-only/dummy1/l1.pdb"

# Path to your PDB file

# Load the PDB file using MDTraj
traj = md.load(pdb_path)

# Visualize using nglview
view = nv.show_mdtraj(traj)

# Add a basic representation (ball-and-stick for atoms and bonds)
view.add_representation("ball+stick", selection="all")

# Display the visualization
view


NGLWidget()

**Inference Data:**

In [7]:
# Parameters
scaling_factor = 30
npz_files = [
    "/home/bfd21/rds/hpc-work/tbg/result_data/Dec-3-2024/bb_all_sc_adj_batch-0.npz",
    "/home/bfd21/rds/hpc-work/tbg/result_data/Dec-3-2024/bb_all_sc_adj_batch-1.npz",
]

# Load PDB and infer bonds
pdb = md.load(pdb_path)
pdb.topology.create_standard_bonds()  # Infer bonds based on atom types and residue connectivity
topology = pdb.topology

# Load and process generated samples
all_samples = []
for npz_file in npz_files:
    data = np.load(npz_file)
    samples_np = data["samples_np"]
    # Reshape samples to match the number of particles and scale them
    n_particles = len(list(topology.atoms))
    samples = samples_np.reshape(-1, n_particles, 3) * scaling_factor
    all_samples.append(samples)

# Combine all samples into a single array
all_samples = np.concatenate(all_samples, axis=0)

# Create an MDTraj trajectory with the inferred bonds
traj = md.Trajectory(
    xyz=all_samples,  # Shape: (n_frames, n_atoms, 3)
    topology=topology
)

# Center the trajectory around the origin
traj.center_coordinates()

# Visualize using nglview
view = nv.show_mdtraj(traj)

# Representations
view.add_representation("cartoon", selection="protein")  # Protein backbone
view.add_representation("ball+stick", selection="all", radius=1)  # Atoms and bonds

# Display the visualization
view


NGLWidget(max_frame=134)

**Real Data**

In [ ]:
# Parameters
scaling_factor = 30
pdb_path = "/home/bfd21/rds/hpc-work/sample_cyclic_md/ligand-only/dummy1/l1.pdb"
npy_files = [
    "/home/bfd21/rds/hpc-work/sample_cyclic_md/ligand-only/dummy1/dummy1_test.npy",
    "/home/bfd21/rds/hpc-work/sample_cyclic_md/ligand-only/dummy1/dummy1_train.npy",
]

# Load PDB and infer bonds
pdb = md.load(pdb_path)
pdb.topology.create_standard_bonds()  # Infer bonds based on atom types and residue connectivity
topology = pdb.topology
n_atoms = len(list(topology.atoms))  # Convert generator to list and measure length

# Load and process generated samples
all_samples = []
for npy_file in npy_files:
    samples_np = np.load(npy_file)
    # Reshape samples to match the number of particles and scale them
    samples = samples_np.reshape(-1, n_atoms, 3) * scaling_factor
    all_samples.append(samples)

# Combine all samples into a single array
all_samples = np.concatenate(all_samples, axis=0)

# Create an MDTraj trajectory with the inferred bonds
traj = md.Trajectory(
    xyz=all_samples,  # Shape: (n_frames, n_atoms, 3)
    topology=topology  # Preserve bonds from the PDB
)

# Center the trajectory around the origin
traj.center_coordinates()

# Visualize using nglview
view = nv.show_mdtraj(traj)

# Representations
view.add_representation("cartoon", selection="protein")  # Protein backbone
view.add_representation("ball+stick", selection="all", radius=0.5)  # Atoms and bonds

# Display the visualization
view


NGLWidget(max_frame=178580)